<a href="https://colab.research.google.com/github/ctrivino1/ML_class_final/blob/main/Chris_Trivino_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [12]:
!pip install pyforest
!pip install git+https://github.com/keras-team/keras-preprocessing.git
import pyforest
import shutil, os
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory

  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-bc5sotwq
  Running command git clone -q https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-bc5sotwq


In [2]:
!git clone https://github.com/ctrivino1/ML_class_final.git

Cloning into 'ML_class_final'...
remote: Enumerating objects: 21410, done.
remote: Counting objects: 100% (4290/4290), done.
remote: Compressing objects: 100% (4287/4287), done.
remote: Total 21410 (delta 0), reused 4290 (delta 0), pack-reused 17120
Receiving objects: 100% (21410/21410), 2.38 GiB | 42.77 MiB/s, done.
Checking out files: 100% (21400/21400), done.


In [13]:
training_dir = "/content/ML_class_final/train_imgs_1"
test_dir = "/content/ML_class_final/test_imgs_1"
df=pd.read_csv("/content/ML_class_final/train_1.csv")
df.label = df.label.astype(str)

<IPython.core.display.Javascript object>

In [14]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=.2,zoom_range=.2,horizontal_flip=True,width_shift_range=0.05,height_shift_range=0.05)


train_generator = train_datagen.flow_from_dataframe(dataframe=df, 
                                         directory=training_dir, 
                                         x_col="image_id", 
                                         y_col="label", 
                                         class_mode="categorical", 
                                         target_size=(591,443), 
                                         batch_size=32,
                                         shuffle=True,
                                         subset="training",
                                         validation_split=.2,
                                         seed=69)






Found 17117 validated image filenames belonging to 5 classes.


In [15]:

valid_generator = train_datagen.flow_from_dataframe(dataframe=df, 
                                         directory=training_dir, 
                                         x_col="image_id", 
                                         y_col="label", 
                                         class_mode="categorical", 
                                         target_size=(800,600), 
                                         batch_size=32,
                                         shuffle=True,
                                         subset="validation",
                                         seed=70)

Found 0 validated image filenames belonging to 5 classes.


In [16]:
test_datagen = ImageDataGenerator(rescale=1 / 255.0)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory=test_dir,
    x_col="image_id",
    target_size=(591,443),
    batch_size=1,
    class_mode=None,
    shuffle=False,
)

Found 0 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning:

Found 17117 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.



In [88]:

base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(591,443,3), pooling="max"
)
#.50 @ 196 epoch's


# efficient B7 model

# Freeze the base model
base_model.trainable = False

# Adds ResNet50 into a model with a output layers with 3 classifaction nodes
model = keras.Sequential([
    base_model,tf.keras.layers.Dropout(.6),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(5,activation="softmax") # number of perceptrons
])
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
optimizer="adam", metrics=['acc'])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(train_generator,
                    validation_data = train_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_steps = valid_generator.n//valid_generator.batch_size,
                    epochs=5)




/usr/local/lib/python3.7/dist-packages/pyforest/__init__.py:29: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/5


InvalidArgumentError: ignored